In [21]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Hypothesis Testing: University Town houses values

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. I will run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

I use the following data files:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (I use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. I will only look at GDP data from the first quarter of 2000 onward.


In [22]:
# I use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [23]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame is:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning have been done:

    1. For "State", I removed characters from "[" to the end.
    2. For "RegionName", when applicable, I removed every character from " (" to the end.'''
    
    import os
    university_file = os.path.join("Resources", "university_towns.txt")
    df = pd.read_fwf(university_file, names=["State"],index = False)
    names_ids = df['State'].str.split('[')
    df['RegionName'] = names_ids.str[0]
    df['State'] = names_ids.str[1]
    df['RegionName'] = df['RegionName'].replace('\s*\(.+', '', regex=True)
    df['State'] = df['State'].replace('\s*\d+]', '', regex=True)
    df = df.replace(np.nan, '', regex=True)
    STNAME=''
    for i in df.index:
        if df.loc[i]['State']=='edit]': STNAME = df.loc[i]['RegionName']#collegeT=df[df['Tag']=='']
        df.loc[i]['State']=STNAME
    df = df[df['State']!=df['RegionName']]
    df = df.reset_index(drop=True)
    return df

get_list_of_university_towns().head()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo


In [24]:
def gdp_df():
    '''Returns a data frame containing the GDP data from 2000 upward'''
    import os
    gdp_file = os.path.join("Resources", "gdplev.xls")
    GDP = ScimEn=pd.read_excel(gdp_file,skiprows=5)
    GDP=GDP.loc[:,['Unnamed: 4','GDP in billions of chained 2009 dollars.1']].rename(columns={'Unnamed: 4':'Quater', 'GDP in billions of chained 2009 dollars.1':'GDP billions of chained 2009'})
    GDP=GDP[GDP['Quater']>='2000q1'].reset_index(drop=True)
    return GDP

def get_recession_start(df):
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    for i in df.index:
        if (df.loc[i]['GDP billions of chained 2009']>df.loc[i+1]['GDP billions of chained 2009']) & (df.loc[i+1]['GDP billions of chained 2009']>df.loc[i+2]['GDP billions of chained 2009']):
            Quat = df.loc[i+1]['Quater']
            break

    return Quat

gdp_2000 = gdp_df()
get_recession_start(gdp_2000)

'2008q3'

In [25]:
def get_recession_end(df):
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''

    GDP = df[df['Quater']>=get_recession_start(df)].reset_index(drop=True)
    for i in GDP.index:
        if (GDP.loc[i]['GDP billions of chained 2009']<GDP.loc[i+1]['GDP billions of chained 2009']) & (GDP.loc[i+1]['GDP billions of chained 2009']<GDP.loc[i+2]['GDP billions of chained 2009']):
            Quat = GDP.loc[i+2]['Quater']
            break

    return Quat

get_recession_end(gdp_2000)

'2009q4'

In [26]:
def get_recession_bottom(df):
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    GDP=df[(df['Quater']>=get_recession_start(df)) & (df['Quater']<=get_recession_end(df))]
    GDPLow=np.min(GDP['GDP billions of chained 2009'])
    GDP_quat = GDP[GDP['GDP billions of chained 2009'] == GDPLow].reset_index(drop=True)
    return GDP_quat.loc[0]['Quater']

get_recession_bottom(gdp_2000)

'2009q2'

In [27]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe is a dataframe with
    columns for 2000q1 through 2016q3, and have a multi-index
    in the shape of ["State","RegionName"].'''
    
    import os
    city_file = os.path.join("Resources", "City_Zhvi_AllHomes.csv")
    df = pd.read_csv(city_file)
    House = df.loc[:,['State','RegionName']]
    years = pd.Series(range(2000,2016))
    years = years.astype(str)
    for year in years:
        quat1 = ['-01','-02','-03']
        quat2 = ['-04','-05','-06']
        quat3 = ['-07','-08','-09']
        quat4 = ['-10','-11','-12']
        yearq1=year+'q1'
        yearq2=year+'q2'
        yearq3=year+'q3'
        yearq4=year+'q4'
        for i in range(3):
            quat1[i]=year+quat1[i]
            quat2[i]=year+quat2[i]
            quat3[i]=year+quat3[i]
            quat4[i]=year+quat4[i]
        House[yearq1]=(df[quat1[0]]+df[quat1[1]]+df[quat1[2]])/3
        House[yearq2]=(df[quat2[0]]+df[quat2[1]]+df[quat2[2]])/3
        House[yearq3]=(df[quat3[0]]+df[quat3[1]]+df[quat3[2]])/3
        House[yearq4]=(df[quat4[0]]+df[quat4[1]]+df[quat4[2]])/3
    House['2016q1']=(df['2016-01']+df['2016-02']+df['2016-03'])/3
    House['2016q2']=(df['2016-04']+df['2016-05']+df['2016-06'])/3
    House['2016q3']=(df['2016-07']+df['2016-08'])/2
    House = House.set_index('State')
    House=House.T
    House = House.rename(columns=states)
    House = House.T
    House = House.set_index('RegionName',append=True)
    return House

convert_housing_data_to_quarters().head()

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,515467,522800,528067,532267,540800,557200,572833,582867,591633,587200
California,Los Angeles,207067,214467,220967,226167,233000,239100,245067,253033,261967,272700,...,498033,509067,518867,528800,538167,547267,557733,566033,577467,584050
Illinois,Chicago,138400,143633,147867,152133,156933,161800,166400,170433,175500,177567,...,192633,195767,201267,201067,206033,208300,207900,206067,208200,212000
Pennsylvania,Philadelphia,53000,53633.3,54133.3,54700,55333.3,55533.3,56266.7,57533.3,59133.3,60733.3,...,113733,115300,115667,116200,117967,121233,122200,123433,126933,128700
Arizona,Phoenix,111833,114367,116000,117400,119600,121567,122700,124300,126533,128367,...,164267,165367,168500,171533,174167,179067,183833,187900,191433,195200


In [28]:
def run_ttest(house_df, gdp_df,confidence):
    '''First this function create new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<confidence (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p is
    equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better is either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    housing = house_df
    GDP = gdp_df
    for i in GDP.index:
        if (GDP.loc[i]['GDP billions of chained 2009']>GDP.loc[i+1]['GDP billions of chained 2009']) & (GDP.loc[i+1]['GDP billions of chained 2009']>GDP.loc[i+2]['GDP billions of chained 2009']):
            Qub4rec = GDP.loc[i]['Quater']
            break
    recbot = get_recession_bottom(gdp_df)
    housing['price_ratio']=housing[Qub4rec]/housing[recbot]
    university = pd.merge(get_list_of_university_towns(), housing, how='inner', right_index=True, left_on=['State','RegionName'])
    university = university.set_index(['State','RegionName'])
    nonuniversity = housing.drop(housing.index.intersection(university.index))
    value, p = ttest_ind(university['price_ratio'].dropna(), nonuniversity['price_ratio'].dropna())
    if p < confidence: different=True
    else: different=False
    if np.mean(university['price_ratio'])>np.mean(nonuniversity['price_ratio']): better = "non-university town"
    else: better = "university town"
    
    return (different, p, better)

my_data = convert_housing_data_to_quarters()
run_ttest(my_data,gdp_2000, 0.01)

(True, 0.0027134586648519788, 'university town')